# FRETpredict tutorial (Hsp90)

In [1]:
import MDAnalysis
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import pandas as pd
import seaborn as sns
from FRET import FRETpredict
import warnings

warnings.filterwarnings('ignore')

### Biological background

Cartoon representation of the open structure of Hsp90 (apoprotein). Chains A and B are colored in red and blue, respectively. On each chain, the residues that will be used for FRET Efficiency calculations are represented as spheres.

![title](Hsp90_chains.png)

Closed structure of Hsp90 (Holo protein, with ATP)

![title](Hsp90_closed_chains.png)

## Single protein structure (Hsp90)

In this part of the tutorial, we will perform FRET Efficiency calculations using only a single residue pair to put the rotamer libraries on.

Select residues to put the rotamer libraries on and the corresponding experimental data file.

In [2]:
res1 = 452
chain_res1 = 'A'
chromophore_1 = 532

res2 = 637
chain_res2 = 'B'
chromophore_2 = 647

# Experimental data to compare our predictions.
exp_data_file = '452_532_637_647N_APO.txt'

Create MDAnalysis.Universe object for the protein (open Hsp90)

In [3]:
u = MDAnalysis.Universe('test_systems/Hsp90/openHsp90.pdb')

Create instance of the FRETpredict class

In [4]:
# Clustering cutoff used for the creation of the rotamer library. It can be 10, 20, or 30.
cutoff = 10

FRET = FRETpredict(protein=u, residues=[res1, res2], temperature=293, 
                   chains=[chain_res1, chain_res2], 
                   donor=chromophore_1, acceptor=chromophore_2, electrostatic=True,
                   libname_1='Alexa {} cutoff{:d}'.format(chromophore_1, cutoff),
                   libname_2='Alexa {} cutoff{:d}'.format(chromophore_2, cutoff), 
                   output_prefix='tutorials/test/E{:d}'.format(cutoff))

Run FRET efficiency calculations

In [5]:
FRET.run()

Retrieve experimental data from file for the comparison

In [6]:
filename = 'test_systems/Hsp90/MDA/histograms/{}'.format(exp_data_file)

if os.path.isfile(filename):
    
        df = pd.read_csv(filename, skiprows=6, nrows=40, header=None, sep='\s+')
        Ex = np.average(df[0], weights=df[1])

Create DataFrame of the data 

In [7]:
results = []

data = pd.read_pickle(r'tutorials/test/E{:d}-data-{:d}-{:d}.pkl'.format(cutoff, res1, res2))
results.append({'res': '{:d}-{:d}'.format(res1, res2),
                'k2': float(data.loc['k2']),
                'Ex': Ex,
                'Es': float(data.loc['Estatic']),
                'Ed': float(data.loc['Edynamic1']),
                'Ed2': float(data.loc['Edynamic2']),
                'diffS': np.abs(Ex-data.loc['Estatic']),
                'diffD': np.abs(Ex-data.loc['Edynamic1']), 
                'diffD2': np.abs(Ex-data.loc['Edynamic2'])})
        
results_df = pd.DataFrame(results).set_index('res')

In [8]:
# Show results
results_df

,conformation,k2,Ex,Es,Ed,Ed2,chi2S,chi2D,chi2D2
res,,,,,,,,,
452-637,open,0.936203,0.592377,0.72088,0.948272,0.985024,0.128502,0.355895,0.392647


**For single frame or structures predictions, the best averaging method is "Static" (Es).**

## Protein trajectory (open conformations)

Select residues to put the rotamer libraries on and the corresponding experimental data file.

In [15]:
res1 = 452
chain_res1 = 'A'
chromophore_1 = 532

res2 = 637
chain_res2 = 'B'
chromophore_2 = 647

# Experimental data to compare our predictions.
exp_data_file = '452_532_637_647N_APO.txt'

Create universe for protein trajectory (3 different open conformations of Hsp90).

In [16]:
u_traj = MDAnalysis.Universe('test_systems/Hsp90/conf.pdb', 
                             'test_systems/Hsp90/Hsp90_open_all.xtc')

Create instance of the FRETpredict class

In [17]:
# Clustering cutoff used for the creation of the rotamer library. It can be 10, 20, or 30.
cutoff = 10

FRET = FRETpredict(protein=u_traj, residues=[res1, res2], temperature=293, 
                   chains=[chain_res1, chain_res2], 
                   donor=chromophore_1, acceptor=chromophore_2,
                   electrostatic=True,
                   libname_1='Alexa {} cutoff{:d}'.format(chromophore_1, cutoff),
                   libname_2='Alexa {} cutoff{:d}'.format(chromophore_2, cutoff), 
                   output_prefix='tutorials/test/E{:d}_traj'.format(cutoff))

Run FRET Efficiency calculations

In [18]:
FRET.run()

Retrieve experimental data from file for the comparison

In [19]:
filename = 'test_systems/Hsp90/MDA/histograms/{}'.format(exp_data_file)

if os.path.isfile(filename):

        df = pd.read_csv(filename, skiprows=6, nrows=40, header=None, sep='\s+')
        Ex = np.average(df[0], weights=df[1])

Create DataFrame for the data 

In [20]:
results_traj = []

data_traj = pd.read_pickle(r'test/E{:d}_traj-data-{:d}-{:d}.pkl'.format(cutoff, res1, res2))
results_traj.append({'res': '{:d}-{:d}'.format(res1, res2),
                     'k2': data_traj.loc['k2', 'Average'],
                     'Ex': Ex,
                     'Es': data_traj.loc['Estatic', 'Average'],
                     'Ed': data_traj.loc['Edynamic1', 'Average'],
                     'Ed2': data_traj.loc['Edynamic2', 'Average'],
                     'diffS': np.abs(Ex-data_traj.loc['Estatic', 'Average']),
                     'diffD': np.abs(Ex-data_traj.loc['Edynamic1', 'Average']), 
                     'diffD2': np.abs(Ex-data_traj.loc['Edynamic2', 'Average'])})
        
results_traj_df = pd.DataFrame(results_traj).set_index('res')

In [21]:
# Show results
results_traj_df

,conformation,k2,Ex,Es,Ed,Ed2,chi2S,chi2D,chi2D2
res,,,,,,,,,
452-637,open,0.901243,0.592377,0.783627,0.864143,0.882688,0.191249,0.271766,0.290311


**For trajectory data or multiple protein structures, the best averaging method is "Dynamic2" (Ed2)**